Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.linear_model import LinearRegression

In [5]:
from sklearn.metrics import r2_score

In [6]:
from sklearn.preprocessing import StandardScaler

## 1. Загрузка данных

ОПИСАНИЕ ДАННЫХ

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.

Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [7]:
clients = pd.read_csv('/datasets/insurance.csv')

In [8]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [9]:
clients.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [10]:
clients.head(40)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


Данные в порядке, предобработка не требуется.

Разделяем исходные данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1 за два этапа.

In [11]:
target = clients['Страховые выплаты'] 
features = clients.drop(columns=['Страховые выплаты'], axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5, random_state=12345)

Проведем масштабирование численных признаков.

In [12]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
features_test = scaler.transform(features_test)

Обучаем модель линейной регрессии и делаем проверку на валидационной и тестовой выборках.

In [13]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
predictions_test = model.predict(features_test)
print(" R2 valid исходная = ", r2_score(target_valid, predictions_valid),'\n', "R2 test исходная = ", r2_score(target_test, predictions_test))

 R2 valid исходная =  0.4133907776507605 
 R2 test исходная =  0.43308225564285185


## 2. Умножение матриц

 **Ответ:**  Качество линейной регрессии не изменится в ситуации, когда признаки умножают на обратимую матрицу.

**Обоснование:** 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Преобразуем: умножаем признаки на матрицу К. w0 не меняется

$$
wk = ((X*K)^T (X*K))^{-1} (X*K)^T y = (K^T*X^T*X*K)^{-1} *K^T*X^T *y = (K^T)^{-1}*(X^T)^{-1}*(K^T)^{-1}*X^{-1}*K^{-1}*K^T*X^T*y = X^{-1}*K^{-1}*y
$$

$$
wk/w =  X^{-1}*K^{-1}*y / (X^T X)^{-1} X^T y = X^{-1}*K^{-1} / (X^T)^{-1}*X^{-1}*X^T  = K^{-1} 
$$

$$
ak = X*K*wk = X*K*w*K^{-1} = Xw
$$

## 3. Алгоритм преобразования

**Алгоритм**

Для того, чтобы защитить данные клиентов страховой компании «Хоть потоп», можно использовать умножение исходных признаков на обратимую матрицу. В этом случае все элементы будут умножены на разные числа, поэтому будет сложно определить исходные данные.

**Обоснование**

Качество линейной регрессии после умножения признаков на обратимую матрицу не изменится.

## 4. Проверка алгоритма

In [14]:
matrix_4 = np.random.normal(size = (4, 4)) #Создадим рандомную матрицу, которая имеет высоту=ширине матрицы наших данных=4.

In [15]:
print(matrix_4)

[[ 0.87443379  0.03103166 -0.19378034  1.55806024]
 [ 0.25980835  0.62518339  1.36527653  1.26718007]
 [-2.59079439 -0.30018136 -0.388179    0.06158722]
 [-0.41623534  1.32657301 -0.15070939  1.17284281]]


In [16]:
matrix_4_inv = np.linalg.inv(matrix_4) # создадим обратную матрицу, чтобы быть уверенными, что она обратимая

In [17]:
train_matrix_inv = np.dot(features_train,matrix_4_inv) # перемножим наши признаки на матрицу
valid_matrix_inv = np.dot(features_valid,matrix_4_inv)
test_matrix_inv = np.dot(features_test,matrix_4_inv)

In [18]:
model = LinearRegression() #обучаем модель и смотрим метрику R2
model.fit(train_matrix_inv, target_train)
predictions_valid = model.predict(valid_matrix_inv)
predictions_test = model.predict(test_matrix_inv)
print(" R2 valid при умножении на обратимую матрицу = ", r2_score(target_valid, predictions_valid),'\n', "R2 test при умножении на обратимую матрицу = ", r2_score(target_test, predictions_test))

 R2 valid при умножении на обратимую матрицу =  0.4133907776507606 
 R2 test при умножении на обратимую матрицу =  0.43308225564285185


## Выводы:
    
    При проверке качество линейной регрессии из sklearn установили, что метрика R2 не отличается до и после преобразования.
    
    R2 valid исходная =  0.4133907776507605 
    R2 test исходная =  0.43308225564285185
    
    R2 valid при умножении на обратимую матрицу =  0.4133907776507526 
    R2 test при умножении на обратимую матрицу =  0.4330822556428515